here i attempt to write a REST2 factory for a canonical openmm.system (i.e. containing `HarmonicBondForce`, `HarmonicAngleForce`, `PeriodicTorsionForce`, `NonbondedForce`)

In [1]:
from openmmtools.testsystems import AlanineDipeptideVacuum

define a vacuum alanine dipeptide

In [2]:
ala = AlanineDipeptideVacuum()
sys = ala.system

In [3]:
from perses.annihilation.rest import RESTTopologyFactory

query the topology

In [4]:
ala.topology
for atom in ala.topology.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ALA)>
<Atom 7 (H) of chain 0 residue 1 (ALA)>
<Atom 8 (CA) of chain 0 residue 1 (ALA)>
<Atom 9 (HA) of chain 0 residue 1 (ALA)>
<Atom 10 (CB) of chain 0 residue 1 (ALA)>
<Atom 11 (HB1) of chain 0 residue 1 (ALA)>
<Atom 12 (HB2) of chain 0 residue 1 (ALA)>
<Atom 13 (HB3) of chain 0 residue 1 (ALA)>
<Atom 14 (C) of chain 0 residue 1 (ALA)>
<Atom 15 (O) of chain 0 residue 1 (ALA)>
<Atom 16 (N) of chain 0 residue 2 (NME)>
<Atom 17 (H) of chain 0 residue 2 (NME)>
<Atom 18 (C) of chain 0 residue 2 (NME)>
<Atom 19 (H1) of chain 0 residue 2 (NME)>
<Atom 20 (H2) of chain 0 residue 2 (NME)>
<Atom 21 (H3) of chain 0 residue 2 (NME)>


I'll make the ALA residue 'solute' for REST2

remove the `CMMForce`

In [5]:
ala.system.removeForce(4)

make the factory

In [6]:
factory = RESTTopologyFactory(ala.system, solute_region=list(range(6, 16)))

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


pull the system

In [8]:
REST_system = factory.REST_system

query the forces to make sure it _looks_ right

In [9]:
REST_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x7f2bcd9fc780> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x7f2bcd9fc3c0> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x7f2bcd9fc360> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f2bcd9fc480> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7f2bcd9fc510> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x7f2bcd9fc4e0> >]

all of the nonbonded forces must have the same number of particles

In [16]:
num_particles = REST_system.getNumParticles()
assert num_particles == factory._out_system_forces['CustomNonbondedForce'].getNumParticles()
assert num_particles == factory._out_system_forces['NonbondedForce'].getNumParticles()

## TODO:

here is a few things that have to be done...

- Given the default Global variables, do energy component bookkeeping with the REST system and the original system. Specifically, the first `CustomBondForce` must match the `HarmonicBondForce` (same with the second and third forces with the `HarmonicAngle` and `PeriodicTorsionForce`s). The sum of the `NonbondedForce`, the `CustomNonbondedForce`, and the `CustomBondForce` (last one) must be equal to the energy of the original system's `NonbondedForce`.

- Create a subclass of the `AlchemicalState` to implement the appropriate parameters for `solute_scale` and `inter_scale` (which are `GlobalParameters` of the `REST_system`) and perform Repex.

- Run REST2!